In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!nvidia-smi

Fri Jul 17 17:29:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    34W / 149W |     11MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 7.0MB/s 


In [8]:
import easydict
import os
import argparse
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn, optim

import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

import torchvision
from torchvision.utils import save_image
from torchvision import datasets, transforms

In [25]:
opt = easydict.EasyDict({
    "n_epochs": 200,
    "batch_size": 128,
    "lr": 0.0002,
    "b1": 0.5,
    "b2": 0.999,
    "n_cpu": 0,
    "latent_dim": 100,
    "n_classes": 10,
    "img_size": 32,
    "channels": 3,
    "sample_size": 500,
    
})
print(opt)


img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False

{'n_epochs': 200, 'batch_size': 128, 'lr': 0.0002, 'b1': 0.5, 'b2': 0.999, 'n_cpu': 0, 'latent_dim': 100, 'n_classes': 10, 'img_size': 32, 'channels': 3, 'sample_size': 500}


In [26]:
class Classifier(nn.Module):
    # initializers
    def __init__(self):
      super(Classifier, self).__init__() 
      self.conv1 = nn.Conv2d(opt.channels, 10, 5)
      self.conv2 = nn.Conv2d(10, 20, 5)
      self.fc1 = nn.Linear(500, 50)
      self.fc2 = nn.Linear(50, 10)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = F.relu(x)
    
        x = self.conv2(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = F.relu(x)
    
        x = x.view(-1, 500)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.sigmoid(x)

def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

In [27]:
class MyDataset(Dataset):

    def __init__(self, data_tensor, target_tensor, transform):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
        self.transform = transform

    def __getitem__(self, index):
        return (self.data_tensor[index], self.target_tensor[index])

    def __len__(self):
        return len(self.data_tensor)

    # transform = transforms.Compose([
    #     transforms.Resize(opt.img_size),                            
    #     transforms.ToTensor(),  
    #     transforms.Normalize([0.5], [0.5])     
    # ])

In [12]:
 def load_data(Name):
    if Name == 'MNIST':
      # Configure data loader
      os.makedirs("/content/drive/My Drive/Summer Project/MNIST", exist_ok=True)
      dataset = datasets.MNIST(
        "/content/drive/My Drive/Summer Project/MNIST",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
      )
    elif Name == "FashionMNIST":
      # Configure data loader
      os.makedirs("/content/drive/My Drive/Summer Project/FashionMNIST", exist_ok=True)
      dataset = datasets.FashionMNIST(
        "/content/drive/My Drive/Summer Project/FashionMNIST",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
      )
    elif Name == "SVHN":
    # Configure data loader
      os.makedirs("/content/drive/My Drive/Summer Project/SVHN", exist_ok=True)
      dataset = datasets.SVHN(
        "/content/drive/My Drive/Summer Project/SVHN",
        split = "train",
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
      )
    elif Name == "CIFAR10":
      os.makedirs("/content/drive/My Drive/Summer Project/CIFAR10", exist_ok=True)
      dataset = datasets.CIFAR10(
          "/content/drive/My Drive/Summer Project/CIFAR10",
          train=True,
          download=True,
          transform=transforms.Compose(
              [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
          ),
      )
    
    dataloader = torch.utils.data.DataLoader(
      dataset,
      batch_size=1,
      shuffle=False,
    )

    index_labels = []
    images = [] 
    for j in range(opt.n_classes):
        y = []
        x = []
        for i, (img, label) in enumerate(dataloader):
            if label == j:
                y.append(i)
                img = img.reshape(opt.channels,32,32)
                x.append(img)
        index_labels.append(y)
        images.append(x)
        
    train_data = []
    train_labels = []

    for i in range(opt.n_classes):
      for j in range(opt.sample_size):
          train_data.append(images[i][j])
          train_labels.append(i)

    mydataset = MyDataset(train_data, train_labels, transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ))
    
    loader = torch.utils.data.DataLoader(
        mydataset,
        batch_size=opt.batch_size,
        shuffle=True,
    )
    return loader


def load_test(Name):
  if Name == "MNIST":
    # Configure data loader
    os.makedirs("/content/drive/My Drive/Summer Project/MNIST", exist_ok=True)
    dataset = datasets.MNIST(
            "/content/drive/My Drive/Summer Project/MNIST",
            train=False,
            download=True,
            transform=transforms.Compose(
                [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
            ),
        )
  elif Name == "FashionMNIST":
    os.makedirs("/content/drive/My Drive/Summer Project/FashionMNIST", exist_ok=True)
    dataset = datasets.FashionMNIST(
          "/content/drive/My Drive/Summer Project/FashionMNIST",
          train=False,
          download=True,
          transform=transforms.Compose(
              [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
          ),
      )
  elif Name == "SVHN":
    os.makedirs("/content/drive/My Drive/Summer Project/SVHN", exist_ok=True)
    dataset = datasets.SVHN(
          "/content/drive/My Drive/Summer Project/SVHN",
          split = "test",
          download=True,
          transform=transforms.Compose(
              [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
          ),
      )
  elif Name == "CIFAR10":
    os.makedirs("/content/drive/My Drive/Summer Project/CIFAR10", exist_ok=True)
    dataset = datasets.CIFAR10(
          "/content/drive/My Drive/Summer Project/CIFAR10",
          train=False,
          download=True,
          transform=transforms.Compose(
              [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
          ),
      )
    
  testloader = torch.utils.data.DataLoader(
      dataset,
      batch_size=64,
      shuffle=False,
  )

  return testloader

In [28]:
# Load Data

dataloader = load_data('SVHN')
testloader = load_test("SVHN")

Using downloaded and verified file: /content/drive/My Drive/Summer Project/SVHN/train_32x32.mat
Using downloaded and verified file: /content/drive/My Drive/Summer Project/SVHN/test_32x32.mat


In [21]:
classifier = Classifier()
optimizer = torch.optim.SGD(classifier.parameters(), lr=0.1)
classifier.train()

Classifier(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=500, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [22]:
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

classification_loss = torch.nn.CrossEntropyLoss()

if cuda:
    classifier.cuda()
    classification_loss.cuda()

In [23]:
for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):
        batch_size = imgs.shape[0]
        # Adversarial ground truths
        optimizer.zero_grad()

        # Loss measures generator's ability to fool the discriminator
        imgs = Variable(imgs.cuda())
        validity = classifier(imgs)

        loss = classification_loss(validity, labels.cuda())

        loss.backward()

        optimizer.step()

    print("Epoch: %d, [loss: %f]"% (epoch, loss.item()))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 0, [loss: 2.308785]
Epoch: 1, [loss: 2.298483]
Epoch: 2, [loss: 2.306787]
Epoch: 3, [loss: 2.309391]
Epoch: 4, [loss: 2.303532]
Epoch: 5, [loss: 2.300651]
Epoch: 6, [loss: 2.304201]
Epoch: 7, [loss: 2.293811]
Epoch: 8, [loss: 2.301433]
Epoch: 9, [loss: 2.305163]
Epoch: 10, [loss: 2.289199]
Epoch: 11, [loss: 2.297144]
Epoch: 12, [loss: 2.271449]
Epoch: 13, [loss: 2.295544]
Epoch: 14, [loss: 2.297136]
Epoch: 15, [loss: 2.287886]
Epoch: 16, [loss: 2.287016]
Epoch: 17, [loss: 2.228431]
Epoch: 18, [loss: 2.240795]
Epoch: 19, [loss: 2.102610]
Epoch: 20, [loss: 2.074453]
Epoch: 21, [loss: 2.041341]
Epoch: 22, [loss: 2.012744]
Epoch: 23, [loss: 1.883748]
Epoch: 24, [loss: 1.872383]
Epoch: 25, [loss: 1.790233]
Epoch: 26, [loss: 1.898191]
Epoch: 27, [loss: 1.811726]
Epoch: 28, [loss: 1.817275]
Epoch: 29, [loss: 1.735111]
Epoch: 30, [loss: 1.700562]
Epoch: 31, [loss: 1.583911]
Epoch: 32, [loss: 1.599715]
Epoch: 33, [loss: 1.627607]
Epoch: 34, [loss: 1.701725]
Epoch: 35, [loss: 1.625853]
Ep

In [24]:
total = 0
acc = 0
classifier.eval()
for i, (imgs, labels) in enumerate(testloader):
    batch_size = imgs.shape[0]
    # Configure input
    real_imgs = Variable(imgs.type(FloatTensor))
    labels = Variable(labels.type(LongTensor))

    validity = classifier(real_imgs)
    result = torch.argmax(validity, axis = 1)
  
    for j in range(batch_size):
      total += 1
      if result[j] == labels[j]:
        acc += 1

print('accuracy: ', acc/total*100)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


accuracy:  76.24462200368777
